## Configuración de ambiente de trabajo

```bash
pip install --upgrade pip
```

```bash
pip install pyjanitor matplotlib==3.5.1 missingno numpy pandas pyreadr seaborn session-info upsetplot==0.6.1
```

or 

```bash
pip install -r requirements.txt
```

### Importar librerías

In [ ]:
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot
from fs import open_fs
from pathlib import Path
import shutil # Necesario para la operación de movimiento de archivos
import sys
import warnings
# Puedes ser más específico con el tipo de advertencia si lo conoces, por ejemplo:
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

# Para las advertencias relacionadas con pkg_resources (como las de tu captura):
warnings.filterwarnings("ignore", message="pkg_resources is deprecated as an API")
warnings.filterwarnings("ignore", message="Deprecated call to pkg_resources.declare_namespace")

from pathlib import Path

# This code snippet is used to import a custom Python module named 'pandas_missing_extension.py'
# located in the parent directory of the current notebook.
# 1. Get the path of the current notebook (where live-exploration-missing-values.ipynb is)
#    This will give you something like:
#    /home/paco/datos_faltantes/curso-datos-faltantes-main/jupyter/live-exploration-missing-values.ipynb
current_notebook_path = Path.cwd() # Path.cwd() returns the current working directory of the kernel

# 2. Navigate up one level to find the directory containing 'pandas_missing_extension.py'
#    This will give you:
#    /home/paco/datos_faltantes/curso-datos-faltantes-main/
project_root_dir = current_notebook_path.parent

# 3. Add this directory to Python's system path if it's not already there
#    This allows Python to find modules in this directory.
if str(project_root_dir) not in sys.path:
    sys.path.append(str(project_root_dir))
    print(f"Added {project_root_dir} to sys.path")

# 4. Now, import your custom accessor module
#    Python will now find 'pandas_missing_extension.py' in the added path.
import pandas_missing_extension

### Configurar el aspecto general de las gráficas del proyecto

In [ ]:
%matplotlib inline

sns.set(
    rc={
        "figure.figsize": (10, 10)
    }
)

sns.set_style("whitegrid")

### Operar con valores faltantes

##### Python

In [ ]:
print(None or True,
      None or True,
      None == None,
      None is None,
      type(None),
      sep='\n')      

#### NumPy

In [ ]:
# print(
#     np.nan  or True,
#     np.nan is np.nan,
#     np.nan == np.nan,
#     np.nan / 2,
#     type(np.nan),
#     np.isnan(np.nan),
#     sep='\n'    
# )

#### Pandas

In [ ]:
# test_missing_df = pd.DataFrame.from_dict(
#     data=dict(
#         x=[0, 1, np.nan, np.nan, None],
#         y=[0, 1, pd.NA, np.nan, None]
#     )
# )
# print(test_missing_df)

In [ ]:
# test_missing_df.isna()

In [ ]:
# test_missing_df.isnull()

In [ ]:
# test_missing_df.x.isnull()

In [ ]:
# pd.Series([1, np.nan])

In [ ]:
# pd.Series([pd.to_datetime('2022-01-01'), np.nan])

In [ ]:
# pd.Series([-1]).isnull()

### Cargar los conjuntos de datos

In [ ]:
# 1️⃣ Crear la carpeta .kaggle en tu home (usuario local)
import os
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# 2️⃣ Descargar el dataset
!kaggle datasets download -d kumargh/pimaindiansdiabetescsv

# 3️⃣ Descomprimir
!yes | unzip -o pimaindiansdiabetescsv.zip -d pima_diabetes



In [ ]:
# --- PARTE 1: UBICAR Y MOVER EL ARCHIVO ---

# 1. Obtener la ruta del directorio donde se está ejecutando este Notebook.
# Esto te dará algo como: Path('/home/paco/datos_faltantes/curso-datos-faltantes-main/jupyter')
current_notebook_dir = Path.cwd()
print(f"Directorio actual del Notebook: {current_notebook_dir}")

# 2. Construir la ruta al archivo de ORIGEN (donde está el CSV después de descomprimir).
# Es 'pima_diabetes/pima-indians-diabetes.csv' RELATIVO al directorio del Notebook.
source_file_path = current_notebook_dir / 'pima_diabetes' / 'pima-indians-diabetes.csv'
print(f"Ruta de origen esperada: {source_file_path}")

# 3. Construir la ruta al directorio de DESTINO ('data/').
# El directorio 'data' está UN NIVEL ARRIBA del directorio del Notebook (jupyter/)
# y luego se entra en 'data/'.
# current_notebook_dir.parent te lleva a 'curso-datos-faltantes-main/'.
# Luego, le añades 'data'.
destination_dir = current_notebook_dir.parent / 'data'
print(f"Directorio de destino esperado: {destination_dir}")

# 4. Construir la ruta COMPLETA del archivo en su destino.
destination_file_path = destination_dir / 'pima-indians-diabetes.csv'
print(f"Ruta de destino final: {destination_file_path}")

# 5. Mover el archivo SÓLO SI EXISTE en el origen.
if source_file_path.exists():
    # shutil.move es como el comando 'mv' de Linux o 'move' de Windows.
    # Mueve el archivo de source_file_path a destination_file_path.
    shutil.move(source_file_path, destination_file_path)
    print(f"\n¡Éxito! Archivo '{source_file_path.name}' movido a '{destination_file_path}'.")
else:
    print(f"\nError: El archivo de origen '{source_file_path}' no fue encontrado.")
    print("Asegúrate de que el archivo 'pimaindiansdiabetescsv.zip' se haya descargado y descomprimido correctamente en 'jupyter/pima_diabetes/'.")

# --- PARTE 2: CARGAR EL ARCHIVO CON PANDAS DESDE LA NUEVA UBICACIÓN ---

# Ahora que el archivo está en 'data/', la ruta para Pandas debe reflejar eso.
# Desde 'jupyter/', para llegar a 'data/', subimos un nivel ('../') y entramos en 'data/'.
final_csv_load_path_for_pandas = '../data/pima-indians-diabetes.csv'

try:
    diabetes_df = pd.read_csv(
    final_csv_load_path_for_pandas,
    sep=",",
    names=[
        "pregnancies",
        "glucose",
        "blood_pressure",
        "skin_thickness",
        "insulin",
        "bmi",
        "diabetes_pedigree_function",
        "age",
        "outcome",
    ]
    )
    print(diabetes_df.head())
    
except FileNotFoundError:
    print(f"\nError: No se pudo cargar el archivo CSV desde '{final_csv_load_path_for_pandas}'.")
    print("Verifica que el archivo haya sido movido correctamente y que la ruta sea correcta.")

### Pima Indians Diabetes

### naniar (oceanbuoys, pedestrian, riskfactors)

#### Crear unidades de información de los conjuntos de datos y descargar y cargar los conjuntos de datos

In [ ]:
# --- Configuración (ajusta si es necesario) ---
# Obtener el directorio padre de tu notebook (curso-datos-faltantes-main/)
# Esto es esencial para ubicar la carpeta 'data' de forma robusta.
project_root_dir = Path.cwd().parent
data_dir = project_root_dir / 'data' # La ruta completa a tu directorio 'data'

# Nombres de los datasets (sin extensión .rda)
# Asegúrate de que estos nombres coincidan con los nombres de los objetos R dentro de los archivos .rda
datasets_names = ["oceanbuoys", "pedestrian", "riskfactors"]
extension = ".rda"

# Diccionario para almacenar los DataFrames
datasets_dfs = {}

# --- Lectura y Conversión ---

print(f"Buscando archivos .rda en: {data_dir}\n")

for dataset_name in datasets_names:
    # Construir la ruta completa del archivo .rda
    dataset_file_path = data_dir / f"{dataset_name}{extension}"

    if dataset_file_path.exists():
        print(f"Leyendo '{dataset_file_path.name}'...")
        try:
            # Leer el archivo .rda
            # .get(dataset_name) intenta extraer un objeto llamado 'dataset_name' del archivo .rda
            df = pyreadr.read_r(str(dataset_file_path)).get(dataset_name)
            datasets_dfs[f"{dataset_name}_df"] = df
            print(f"  -> '{dataset_name}_df' creado con éxito.")
        except Exception as e:
            print(f"  Error al leer '{dataset_file_path.name}' con pyreadr: {e}")
            print(f"  Asegúrate de que el nombre del objeto R dentro del archivo .rda sea '{dataset_name}'.")
            print("  Si el nombre del objeto R es diferente al nombre del archivo, necesitarás ajustar .get().")
    else:
        print(f"Advertencia: Archivo '{dataset_file_path.name}' no encontrado en '{data_dir}'. Saltando.")

# --- Verificación ---
print("\n--- DataFrames cargados ---")
if datasets_dfs:
    for df_name, df in datasets_dfs.items():
        print(f"\nDataFrame: {df_name}")
        print(df.head())
        print(f"Forma: {df.shape}")
else:
    print("No se cargaron DataFrames.")


#### Incluir conjuntos de datos en nuestro ambiente local

In [ ]:
# --- Mover DataFrames del Diccionario al Ámbito Global y Limpiar ---

print("\n--- Convirtiendo DataFrames de diccionario a variables globales ---")
if datasets_dfs:
    # Esta línea es la clave: extrae cada par clave-valor del diccionario
    # y los establece como variables independientes en tu entorno de Jupyter.
    locals().update(**datasets_dfs)
    print("DataFrames movidos al ámbito global (ej. 'oceanbuoys_df').")

    # Una vez que los DataFrames están como variables globales,
    # el diccionario datasets_dfs ya no es necesario y se puede eliminar para liberar memoria.
    del datasets_dfs
    print("Diccionario 'datasets_dfs' eliminado.")
else:
    print("No hay DataFrames cargados para mover.")

# --- Verificación de Variables Globales ---

print("\n--- Verificación de acceso a DataFrames como variables globales ---")
# Ahora puedes acceder a los DataFrames directamente por sus nombres
# Por ejemplo:
try:
    if 'oceanbuoys_df' in locals():
        print("Variable 'oceanbuys_df' existe.")
        print(oceanbuoys_df.head())
        print(f"Forma de 'oceanbuoys_df': {oceanbuoys_df.shape}")
    else:
        print("La variable 'oceanbuoys_df' no se pudo crear (verifica nombres de archivos/objetos R).")

    if 'pedestrian_df' in locals():
        print("Variable 'pedestrian_df' existe.")
        print(pedestrian_df.head())
        print(f"Forma de 'pedestrian_df': {pedestrian_df.shape}")

    if 'oceanbuoys_df' in locals():
        print("Variable 'oceanbuoys_df' existe.")
        print(oceanbuoys_df.head())
        print(f"Forma de 'oceanbuoys_df': {oceanbuoys_df.shape}")

except NameError as e:
    print(f"\nError al intentar acceder a una variable global: {e}")
    print("Esto puede ocurrir si el nombre del objeto R dentro del archivo .rda no coincide con el 'dataset_name' utilizado en la lista.")
except Exception as e:
    print(f"\nOcurrió un error inesperado durante la verificación: {e}")

### Verificar carga

In [ ]:
oceanbuoys_df.shape, pedestrian_df.shape, riskfactors_df.shape

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Set to a large width to avoid wrapping
pd.set_option('display.max_info_columns', 1000) # Keep this as a large int

print("--- Displaying info for oceanbuoys_df ---")
oceanbuoys_df.info()

print("\n--- Confirming DataFrame dimensions ---")
print(f"Number of rows: {oceanbuoys_df.shape[0]}")
print(f"Number of columns: {oceanbuoys_df.shape[1]}") # This will tell you the actual count

print("\n--- Listing all column names ---")
# This is the most definitive way to see all column names.
for i, col in enumerate(oceanbuoys_df.columns):
    print(f"Column {i}: {col}")

## Tabulación de valores faltantes

In [ ]:
riskfactors_df.isna()

### Resúmenes básicos de valores faltantes

In [ ]:
riskfactors_df.shape

#### Número total de valores completos (sin observaciones faltantes)

In [ ]:
riskfactors_df.missing.number_complete()

#### Número total de valores faltantes

In [ ]:
riskfactors_df.missing.number_missing()

### Resúmenes tabulares de valores faltantes

#### Variables / Columnas

###### Resumen por variable

In [ ]:
riskfactors_df.missing.missing_variable_table()

###### Tabulación del resumen por variable

In [ ]:
riskfactors_df.missing.missing_variable_table()

#### Casos / Observaciones / Filas

###### Resúmenes por caso

In [ ]:
riskfactors_df.missing.missing_case_table()

###### Tabulación del resumen por caso

In [ ]:
riskfactors_df.missing.missing_case_table()

###### Intervalos de valores faltantes

In [ ]:
riskfactors_df.missing.missing_variable_span(variable='weight_lbs', span_every= 50)

###### _Run length_ de valores faltantes

In [ ]:
riskfactors_df.missing.missing_variable_run(variable='weight_lbs')

### Visualización inicial de valores faltantes

#### Variable

In [ ]:
riskfactors_df.missing.missing_variable_plot()

### Casos / Observaciones / Filas

In [ ]:
riskfactors_df.missing.missing_case_plot()

In [ ]:
(riskfactors_df
 .missing
 .missing_variable_span_plot(
   variable='weight_lbs',
    span_every=20
    )
)

In [ ]:
missingno.bar(riskfactors_df, figsize=(10, 5), fontsize=9)

In [ ]:
missingno.matrix(riskfactors_df, figsize=(20, 10), fontsize=11)

In [ ]:
warnings.filterwarnings("ignore")
(riskfactors_df
    .missing
    .missing_upsetplot(
      variables=None,
      element_size=60
    )
)

_The UpSet plot visualizes the intersections of missing values across multiple variables (or columns), showing which combinations of them have concurrent absent data and their frequencies._

In [ ]:
warnings.filterwarnings("ignore")
(riskfactors_df
    .missing
    .missing_upsetplot(
      variables=['weight_lbs', 'pregnant', 'smoke_stop'],
      element_size=60
    )
)

The UpSet plot reveals that in the given dataset, the most common missing data pattern occurs when **`smoke_stop` and `pregnant` are concurrently absent** in over 150 instances, while other combinations of `weight_lbs`, `smoke_stop`, and `pregnant` are less frequent.

## Codificación de valores faltantes

<div class="alert alert-warning", role="alert">
    <b style="font-size: 1.5em;">🚧 Advertencia</b>
    <p>
    Al igual que cada persona es una nueva puerta a un mundo diferente, los <b>valores faltantes</b> existen en diferentes formas y colores. Al trabajar con valores faltantes será crítico entender sus distintas representaciones. A pesar de que el conjunto de datos de trabajo pareciera que no contiene valores faltantes, deberás ser capaz de ir más allá de lo observado a simple vista para remover el manto tras el cual se esconde lo desconocido.
    </p>
</div>

### Valores comúnmente asociados a valores faltantes

#### Cadenas de texto

In [ ]:
common_na_strings = (
    "missing",
    "NA",
    "N A",
    "N/A",
    "#N/A",
    "NA ",
    " NA",
    "N /A",
    "N / A",
    " N / A",
    "N / A ",
    "na",
    "n a",
    "n/a",
    "na ",
    " na",
    "n /a",
    "n / a",
    " a / a",
    "n / a ",
    "NULL",
    "null",
    "",
    "?",
    "*",
    ".",
)

#### Números

In [ ]:
common_na_numbers = (-9, -99, -999, -9999, 9999, 66, 77, 88, -1)

### ¿Cómo encontrar los valores comúnmente asociados a valores faltantes?

In [ ]:
missing_data_example_df = pd.DataFrame.from_dict(
    dict(
        x = [1, 3, "NA", -99, -98, -99],
        y = ["A", "N/A", "NA", "E", "F", "G"],
        z = [-100, -99, -98, -101, -1, -1]
    )
)

missing_data_example_df

In [ ]:
missing_data_example_df.missing.number_missing()

#### Revisar tipos de datos

In [ ]:
missing_data_example_df.dtypes

#### Revisar valores únicos de los datos

In [ ]:
missing_data_example_df.x.unique()

In [ ]:
(
  missing_data_example_df
  .select_dtypes(object)
  .apply(pd.unique)
)

### Sustituyendo valores comúnmente asociados a valores faltantes

#### Sustitución desde la lectura de datos

In [ ]:
pd.read_csv(
    '../data/missing_data_enconding_example.csv',
    na_filter=True,
    na_values=[-99, -1]
)

#### Sustitución global

In [ ]:
(
  missing_data_example_df
  .replace(
    to_replace=[-1, -98, -99, "NA", "N/A", "n/a", "na", "n a", "N A", "N /A", "N / A", "N / A ", "n /a", "n / a", " n / a", "n / a ", "", "*", ".", "?"],
    value=np.nan
  )
)

#### Sustitución dirigida

### Conversión de valores faltantes implícitos a explícitos

<div class="alert alert-warning", role="alert">
    <b style="font-size: 1.5em;">🚧 Advertencia</b>
    <br>
    <br>
    <p>
        <i>
        "<b>Implícito</b> se refiere a todo aquello que se entiende que está incluido
        pero sin ser expresado de forma directa o explícitamente."
        </i>
    </p>
    <p>
    Un <code>valor faltante implícito</code> indica que el valor faltante <b>debería estar incluido</b>
    en el conjunto de datos del análisis, <b>sin que éste lo diga</b> o lo <b>especifique</b>.
    Por lo general, son valores que podemos encontrar al pivotar nuestros datos
    o contabilizar el número de apariciones de combinaciones de las variables de estudio.
    </p>
</div>

In [ ]:
implicit_to_explicit_df = pd.DataFrame.from_dict(
    data={
        "name": ["lynn", "lynn", "lynn", "zelda"],
        "time": ["morning", "afternoon", "night", "morning"],
        "value": [350, 310, np.nan, 320]
    }
)

implicit_to_explicit_df

### Estrategias para la identificación de valores faltantes implícitos

#### Pivotar la tabla de datos

In [ ]:
(
  implicit_to_explicit_df
  .pivot_wider(
    index='name',
    names_from='time',
    values_from='value'
  )
)

#### Cuantificar ocurrencias de n-tuplas

In [ ]:
(implicit_to_explicit_df
 .value_counts(
   subset=['name'],
    )
  .reset_index(name='n')
  .query('n < 3')
)

### Exponer filas faltantes implícitas a explícitas

<div class="alert alert-info">
    <b style="font-size: 1.5em;">📘 Información</b>
    <p>
       <a href="https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.complete.complete", class="alert-link"><code>janitor.complete()</code></a> está modelada a partir de la función <a href="https://tidyr.tidyverse.org/reference/complete.html", class="alert-link"><code>complete()</code></a> del paquete <a href="https://tidyr.tidyverse.org/index.html", class="alert-link"><code>tidyr</code></a> y es un <i>wrapper</i> alrededor de <a href="https://pyjanitordevs.github.io/pyjanitor/api/functions/#janitor.functions.expand_grid.expand_grid", class="alert-link"><code>janitor.expand_grid()</code></a>, <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html", class="alert-link"><code>pd.merge()</code></a> y <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html", class="alert-link"><code>pd.fillna()</code></a>. En cierto modo, es lo contrario de <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html", class="alert-link"><code>pd.dropna()</code></a>, ya que expone implícitamente las filas que faltan.
    </p>
    <p>
    Son posibles combinaciones de nombres de columnas o una lista/tupla de nombres de columnas, o incluso un  diccionario de nombres de columna y nuevos valores.
    </p>
    <p>
    Las columnas <a href="https://pandas.pydata.org/docs/user_guide/advanced.html"><code>MultiIndex</code></a> no son complatibles.
    </p>
</div>

#### Exponer n-tuplas de valores faltantes

Ejemplo, encontrar los pares faltantes de `name` y `time`.

In [ ]:
print("\n--- Implicit to Explicit DataFrame ---")
print("This DataFrame has implicit missing values (NaN) in the 'value' column.")
print(implicit_to_explicit_df)

print("\n--- Pivoting with NaN values ---")
pivot_nan= (
  implicit_to_explicit_df
  .pivot_wider(
    index='name',          # Datos que se usarán como índice en el nuevo DataFrame
    names_from='time',     # Columnas del nuevo df
    values_from='value',   # Valores que se usarán para llenar las celdas
  )
)
print(pivot_nan)

print("\n--- Using complete to fill NaN values ---")
complete_nan = (
  implicit_to_explicit_df
  .complete(
    'name',
    'time'
  )
)
print(complete_nan)

#### Limitar la exposición de n-tuplas de valores faltantes

In [ ]:
(
  implicit_to_explicit_df
  .complete(
    {'name':['lynn', 'zelda']},
    {'time':['morning', 'afternoon']},
    sort=True   
  )
)

#### Rellenar los valores faltantes

#### Limitar el rellenado de valores faltantes implícitos

## Tipos de valores faltantes

In [ ]:
# diabetes_df.missing.missing_variable_plot()

In [ ]:
diabetes_df[diabetes_df.columns[1:6]] = diabetes_df[diabetes_df.columns[1:6]].replace(0, np.nan)
diabetes_df.missing.missing_variable_plot()

### _Missing Completely At Random_ (MCAR)

In [ ]:
(
    diabetes_df
    .missing.sort_variables_by_missingness()
    .pipe(missingno.matrix)
)

### _Missing At Random_ (MAR)

In [ ]:
(
    diabetes_df
    .missing.sort_variables_by_missingness()
    .sort_values(by='blood_pressure')
    .pipe(missingno.matrix)
)

### _Missing Not At Random_ (MNAR)

In [ ]:
(
    diabetes_df
    .missing.sort_variables_by_missingness()
    .sort_values(by='insulin')
    .pipe(missingno.matrix)
)

Video a ver: [Mecanismos de datos faltantes de YouTube](https://www.youtube.com/watch?v=ARwHkq4t2q0)

## Concepto y aplicación de la matriz de sombras (_i.e._, _shadow matrix_)

 ### Construcción de la matriz de sombras

In [ ]:
riskfactors_df

In [ ]:
(
    riskfactors_df
    .isna()
    .replace({
        False: 'Not missing',
        True: 'Missing'
    })
    .add_suffix('_NA')
    .pipe(
        lambda shadow_matrix: pd.concat(
            [riskfactors_df, shadow_matrix],
            axis='columns'
        )
    )
)

In [ ]:
(
  riskfactors_df
  .missing
  .bind_shadow_matrix(only_missing = True)
  .groupby(["weight_lbs_NA"])["age"]
  .describe()
  .reset_index()
)

In [ ]:
(
    riskfactors_df
    .missing.bind_shadow_matrix(only_missing=True)
    .pipe(
      lambda df : (
        sns.boxenplot(
          data = df,
          x = 'weight_lbs_NA',
          y = 'age'
        )
      )
    )
)

_Utilizar función de utilería 'bind_shadow_matrix()'_

In [ ]:
(
    riskfactors_df
    .missing
    .bind_shadow_matrix(only_missing=True)
)

_Exploracion estadísticos usando nuevas columnas de matriz de sombras_

In [ ]:
(
    riskfactors_df
    .missing
    .bind_shadow_matrix(only_missing=True)
)

_Visualización de valores faltantes en una variable_

In [ ]:
(
    riskfactors_df
    .missing.bind_shadow_matrix(only_missing=True)
    .pipe(
        lambda df : (
            sns.boxenplot(
                data=df,
                x='weight_lbs_NA',
                y='age'
            )
        )
    )
)

In [ ]:
(    
	riskfactors_df    
	.missing    
	.bind_shadow_matrix(only_missing=True)    
	.pipe(        
		lambda df:(           
			sns.displot(                
				data = df,                
				x = 'age',                
				kind = 'kde',                
				hue = 'pregnant_NA',  
				fill = True
			)            
		)    
	)
)

_Shadow matrix_

In [ ]:
(    
	riskfactors_df    
	.missing    
	.bind_shadow_matrix(only_missing=True)    
	.pipe(        
		lambda df:(           
			sns.displot(                
				data = df,                
				x = 'age',                
				col = 'weight_lbs_NA'
			)            
		)    
	)
)

In [ ]:
(    
	riskfactors_df    
	.missing    
	.bind_shadow_matrix(only_missing=True)    
	.pipe(        
		lambda df:(           
			sns.displot(                
				data = df,                
				x = 'age',                
				col = 'pregnant_NA',
                facet_kws={
                    'sharey': False
                }
			)
		)
	)
)

In [ ]:
(    
	riskfactors_df    
	.missing    
	.bind_shadow_matrix(only_missing=True)    
	.pipe(        
		lambda df:(           
			sns.displot(                
				data = df,                
				x = 'age',                
                col = 'marital_NA',
                row = 'weight_lbs_NA'
			)
		)
	)
)

## Visualización de valores faltantes en dos variables

In [ ]:
def column_fill_with_dummies(
    column: pd.Series,
    proportion_below: float = 0.10,  # % del mínimo real para desplazar el piso de dummies
    jitter: float = 0.075,            # fracción del rango para dispersar los dummies
    seed: int = 42                    # semilla para reproducibilidad del ruido
) -> pd.Series:
    """
    Reemplaza los NaN de una serie por puntos 'dummy' numéricos justo por debajo
    del mínimo real, con un pequeño jitter para evitar solapamientos en la gráfica.

    Args:
        column (pd.Series): Serie original con posibles NaN.
        proportion_below (float): Fracción del mínimo real para calcular el piso.
        jitter (float): Fracción del rango para crear dispersión aleatoria.
        seed (int): Semilla para el generador de números aleatorios.

    Returns:
        pd.Series: Nueva serie con los NaN sustituidos por dummies numéricos.
    """

    # 1) Crear copia profunda para no alterar la serie original
    column = column.copy(deep=True)

    # 2) Identificar posiciones con NaN
    missing_mask = column.isna()           # Serie booleana: True donde había NaN
    number_missing_values = missing_mask.sum()  # Cantidad total de NaN

    # 3) Calcular rango auténtico de la serie (max – min)
    real_min = column.min()                # Mínimo real (sin NaN)
    real_max = column.max()                # Máximo real
    column_range = real_max - real_min     # Diferencia para escalar el jitter

    # 4) Definir 'piso' de dummies: un poco por debajo del mínimo real
    #    Ejemplo: con proportion_below=0.10, shift = real_min – 10%·real_min
    column_shift = real_min - (real_min * proportion_below)

    # 5) Generar ruido (jitter) para cada dummy
    #    - np.random.seed fija la semilla para reproducibilidad
    #    - np.random.rand(n) crea n valores en [0,1)
    #    - restar 2 pasa esos valores a [-2, -1) ⇒ desplazamiento negativo garantizado
    #    - multiplicar por column_range * jitter ajusta la magnitud del ruido
    np.random.seed(seed)
    column_jitter = (np.random.rand(number_missing_values) - 2) * column_range * jitter

    # 6) Asignar a cada posición NaN un valor dummy = piso + su jitter correspondiente
    column[missing_mask] = column_shift + column_jitter

    # 7) Devolver la serie modificada (mismos índices, dummies en reemplazo de NaN)
    return column


## Correlación de nulidad

## Eliminación de valores faltantes

<div class="alert alert-warning", role="alert">
    <b style="font-size: 1.5em;">🚧 Advertencia</b>
    <p>
    La eliminación de valores faltantes <b>asume</b> que los valores faltantes están perdidos
    completamente al azar (<code>MCAR</code>). En cualquier otro caso, realizar una
    eliminación de valores faltantes podrá ocasionar <b>sesgos</b> en los
    análisis y modelos subsecuentes.
    </p>
</div>

Primero observa el número total de observaciones y variables que tiene tu conjunto de datos.

### _Pairwise deletion_ (eliminación por pares)

### _Listwise Deletion or Complete Case_ (Eliminación por lista o caso completo)

#### Con base en 1 columna

#### Con base en 2 o más columnas

### Representación gráfica tras la eliminación de los valores faltantes

## Imputación básica de valores faltantes

### Imputación con base en el contexto

In [ ]:
implicit_to_explicit_df = pd.DataFrame(
    data={
        "name": ["lynn", np.nan, "zelda", np.nan, "shadowsong", np.nan],
        "time": ["morning", "afternoon", "morning", "afternoon", "morning", "afternoon",],
        "value": [350, 310, 320, 350, 310, 320]
    }
)

implicit_to_explicit_df

### Imputación de un único valor

## Continúa aprendiendo sobre el manejo de valores faltantes

<div class="alert alert-success">
    <b style="font-size: 1.5em;">✅ ¡Felicidades por terminar el curso!</b>
    <p>
Has aprendido bastante sobre la exploración y manipulación de valores faltantes.
    </p>
    <p>
Empezaste conociento las principales operaciones al trabajar con valores faltantes. Ahora, eres consciente de que estas operaciones no son universales y cada software decide tratar a los valores faltantes a su conveniencia.
    </p>
    <p>
Y, hablando de conveniencias, comenzaste tu camino en la exploración de valores faltantes a través de una representación universal de qué es lo que faltaba. No obstante, no pasó mucho para darte cuenta de que los valores faltantes pueden existir en formas muy variables. Incluso, en formas en las que no sabemos que nos faltan estos valores en sí mismos. 
    </p>
    <p>
Con los valores faltantes ya expuestos, te conviertes en una persona capaz de explorarlos en profundidad de forma estadística y visual. Entendiendo así, los distintos mecanismos que pueden tener los valores faltantes: MCAR, MAR y MNAR.
    </p>
    <p>
A su vez, aprendiste las bases sobre cómo tratarlos a través de la eliminación de elementos o la imputación de valores de una forma básica y sencilla. Por lo tanto, necesitarás continuar tu camino de aprendizaje con un curso que te permita profundizar en estas técnicas de tratamiento para valores faltantes.
    </p>
    <p>
Te recomiendo continuar con mi <a href="https://platzi.com/cursos/datos-faltantes-imputacion/">Curso de Manejo de Datos Faltantes: Imputación</a>. Estoy seguro de que tus habilidades adquiridas hasta el momento mejorarán, permitiéndote realizar análisis cada vez más complejos y cercanos al mundo real.
    </p>
    <p>
    Con mucha alegría por tu logro,
   Jesús Vélez Santiago
    </p>
    
</div>

## Información de sesión

In [ ]:
session_info.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=694a3d08-7f18-421d-9e2f-c2820a79680e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>